In [14]:
import pandas as pd

# Carichiamo il dataset iniziale
file_path = 'actorfilms.csv'
actor_films_df = pd.read_csv(file_path)

# Filtriamo gli attori che hanno partecipato ai film degli Avengers
avengers_films = [
    "The Avengers", "Avengers: Age of Ultron", "Avengers: Infinity War", "Avengers: Endgame"
]
filtered_avengers_df = actor_films_df[
    (actor_films_df['Film'] == "The Avengers") & (actor_films_df['Year'] == 2012) |
    actor_films_df['Film'].isin(avengers_films)
]

# Filtriamo il dataset per contenere solo gli attori presenti nei film degli Avengers
actors_in_avengers = filtered_avengers_df['Actor'].unique()
filtered_dataset = actor_films_df[actor_films_df['Actor'].isin(actors_in_avengers)]

# Creiamo le connessioni tra attori
df = pd.merge(filtered_dataset, filtered_dataset, 'inner', on='FilmID')
df = df[df['ActorID_x'] != df['ActorID_y']]  # Rimuoviamo auto-connessioni

df = df[~df['Film_x'].isin(avengers_films)]

# Creiamo una coppia ordinata (Source, Target) per rimuovere duplicati
df['pair'] = df.apply(lambda row: tuple(sorted([row['Actor_x'], row['Actor_y']])), axis=1)

# Raggruppiamo per coppie e calcoliamo i pesi
grouped = df.groupby('pair').size().reset_index(name='count')
grouped[['Source', 'Target']] = pd.DataFrame(grouped['pair'].tolist(), index=grouped.index)
grouped['weight'] = grouped['count']

# Manteniamo solo le colonne finali
final_dataset = grouped[['Source', 'Target', 'weight']]

# Salviamo il dataset finale (opzionale, se serve)
final_dataset.to_csv('dataset_con_peso.csv', index=False)


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Carichiamo il dataset con peso
dataset_path = 'dataset_con_peso.csv'
df = pd.read_csv(dataset_path)

# Creiamo il grafo non orientato
G = nx.Graph()

# Aggiungiamo i nodi e gli archi con il relativo peso
for _, row in df.iterrows():
    source = row['Source']
    target = row['Target']
    weight = row['weight']
    G.add_edge(source, target, weight=weight)

# Creiamo il layout Spring
pos = nx.spring_layout(G, seed=42)  # seed per rendere il layout riproducibile

plt.figure(figsize=(12, 10))
# Disegno dei nodi con dimensione ridotta
nx.draw_networkx_nodes(G, pos, node_size=20, node_color='skyblue')
# Disegno degli archi
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.7)
# Non disegniamo le etichette

plt.title("Social Network Analysis degli attori")
plt.axis('off')
plt.show()



In [ ]:
import pandas as pd
import networkx as nx

# Carichiamo il dataset con peso
dataset_path = 'dataset_con_peso.csv'
df = pd.read_csv(dataset_path)

# Costruiamo il grafo non orientato partendo dalle colonne Source e Target
G = nx.Graph()
for _, row in df.iterrows():
    source = row['Source']
    target = row['Target']
    weight = row['weight']
    G.add_edge(source, target, weight=weight)

# Calcoliamo le metriche del grafo
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
density = nx.density(G)
is_connected = nx.is_connected(G)

# Raggio, diametro e periferia sono definiti solo se il grafo è connesso
if is_connected:
    radius = nx.radius(G)
    diameter = nx.diameter(G)
    periphery_nodes = nx.periphery(G)
    num_periphery = len(periphery_nodes)
else:
    radius = None
    diameter = None
    num_periphery = None

avg_clustering = nx.average_clustering(G)

# Creiamo la tabella delle informazioni sul grafo
summary_data = {
    "Numero nodi": [num_nodes],
    "Numero archi": [num_edges],
    "Densità": [density],
    "Connesso": [is_connected],
    "Raggio": [radius],
    "Diametro": [diameter],
    "Periferia (#nodi)": [num_periphery],
    "Avg clustering": [avg_clustering]
}
summary_df = pd.DataFrame(summary_data)

# Creiamo una tabella con la top 10 archi per peso
edges_data = []
for u, v, data in G.edges(data=True):
    edges_data.append({"attore1": u, "attore2": v, "wheight": int(data.get("weight", 1))})

edges_df = pd.DataFrame(edges_data)
top10_edges_df = edges_df.sort_values(by="wheight", ascending=False).head(10)

# Mostriamo le tabelle
print("Info Grafo:")
print(summary_df.to_string(index=False))
print("\nTop 10 Archi per Peso:")
print(top10_edges_df.to_string(index=False))


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Carichiamo il dataset con il peso degli archi
dataset_path = 'dataset_con_peso.csv'
df = pd.read_csv(dataset_path)

# Costruiamo il grafo non orientato utilizzando le colonne "Source" e "Target"
G = nx.Graph()
for _, row in df.iterrows():
    source = row['Source']
    target = row['Target']
    weight = row['weight']
    G.add_edge(source, target, weight=weight)

# Calcoliamo la degree centrality per ogni nodo
deg_centrality = nx.degree_centrality(G)

# Creazione del layout Spring
pos = nx.spring_layout(G, seed=42)

# Impostiamo una dimensione fissa per i nodi (ad esempio, 200)
node_size_fixed = 50

# Prepariamo i valori per la colorazione (mappa di calore) basata sulla degree centrality
node_color_values = [deg_centrality[node] for node in G.nodes()]

plt.figure(figsize=(12, 10))
nodes = nx.draw_networkx_nodes(
    G, pos,
    node_size=node_size_fixed,
    node_color=node_color_values,  # Colorazione basata sulla degree centrality
    cmap=plt.cm.hot,              # Puoi scegliere altre colormaps come 'viridis', 'plasma', ecc.
    alpha=0.9
)
nx.draw_networkx_edges(G, pos, alpha=0.7)
# Non disegnare le etichette dei nodi
plt.title("Grafo con Degree Centrality - Mappa di Calore (Nodi a dimensione fissa)")
plt.axis('off')
plt.colorbar(nodes, label="Degree Centrality")
plt.show()


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Carichiamo il dataset con il peso degli archi
dataset_path = 'dataset_con_peso.csv'
df = pd.read_csv(dataset_path)

# Costruiamo il grafo non orientato utilizzando le colonne "Source" e "Target"
G = nx.Graph()
for _, row in df.iterrows():
    source = row['Source']
    target = row['Target']
    weight = row['weight']
    G.add_edge(source, target, weight=weight)

# Calcoliamo la betweenness centrality per ogni nodo
bet_centrality = nx.betweenness_centrality(G)

# Creazione del layout Spring
pos = nx.spring_layout(G, seed=42)

# Impostiamo una dimensione fissa per i nodi
node_size_fixed = 50

# Prepariamo i valori per la colorazione basata sulla betweenness centrality
node_color_values = [bet_centrality[node] for node in G.nodes()]

plt.figure(figsize=(12, 10))
nodes = nx.draw_networkx_nodes(
    G, pos,
    node_size=node_size_fixed,
    node_color=node_color_values,
    cmap=plt.cm.hot,   # Puoi sperimentare con altre colormaps come 'viridis', 'plasma', etc.
    alpha=0.9
)
nx.draw_networkx_edges(G, pos, alpha=0.7)
# Non disegnare le etichette dei nodi
plt.title("Grafo con Betweenness Centrality - Mappa di Calore (Nodi a dimensione fissa)")
plt.axis('off')
plt.colorbar(nodes, label="Betweenness Centrality")
plt.show()

# Opzionale: stampa la betweenness centrality ordinata (decrescente)
print("Betweenness Centrality:")
for node, centrality in sorted(bet_centrality.items(), key=lambda x: x[1], reverse=True):
    print(f"{node}: {centrality:.4f}")


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Carichiamo il dataset con il peso degli archi
dataset_path = 'dataset_con_peso.csv'
df = pd.read_csv(dataset_path)

# Costruiamo il grafo non orientato utilizzando le colonne "Source" e "Target"
G = nx.Graph()
for _, row in df.iterrows():
    source = row['Source']
    target = row['Target']
    weight = row['weight']
    G.add_edge(source, target, weight=weight)

# Calcoliamo la closeness centrality per ogni nodo
closeness_centrality = nx.closeness_centrality(G)

# Creazione del layout Spring
pos = nx.spring_layout(G, seed=42)

# Impostiamo una dimensione fissa per i nodi
node_size_fixed = 50

# Prepariamo i valori per la colorazione basata sulla closeness centrality
node_color_values = [closeness_centrality[node] for node in G.nodes()]

plt.figure(figsize=(12, 10))
nodes = nx.draw_networkx_nodes(
    G, pos,
    node_size=node_size_fixed,
    node_color=node_color_values,
    cmap=plt.cm.hot,
    alpha=0.9
)
nx.draw_networkx_edges(G, pos, alpha=0.7)
plt.title("Grafo con Closeness Centrality - Mappa di Calore (Nodi a dimensione fissa)")
plt.axis('off')
plt.colorbar(nodes, label="Closeness Centrality")
plt.show()

# Opzionale: stampa closeness centrality ordinata
print("Closeness Centrality:")
for node, centrality in sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True):
    print(f"{node}: {centrality:.4f}")


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Carichiamo il dataset con il peso degli archi
dataset_path = 'dataset_con_peso.csv'
df = pd.read_csv(dataset_path)

# Costruiamo il grafo non orientato utilizzando le colonne "Source" e "Target"
G = nx.Graph()
for _, row in df.iterrows():
    source = row['Source']
    target = row['Target']
    weight = row['weight']
    G.add_edge(source, target, weight=weight)

# Calcolo dell'eigenvector centrality
# Usando 'weight' come attributo per il calcolo
eigenvector_centrality = nx.eigenvector_centrality(G, weight='weight', max_iter=1000)

# Creazione del layout Spring
pos = nx.spring_layout(G, seed=42)

# Dimensione fissa dei nodi
node_size_fixed = 50

# Colori dei nodi in base all'eigenvector centrality
node_color_values = [eigenvector_centrality[node] for node in G.nodes()]

plt.figure(figsize=(12, 10))
nodes = nx.draw_networkx_nodes(
    G, pos,
    node_size=node_size_fixed,
    node_color=node_color_values,
    cmap=plt.cm.hot,
    alpha=0.9
)
nx.draw_networkx_edges(G, pos, alpha=0.7)
plt.title("Grafo con Eigenvector Centrality - Mappa di Calore (Nodi a dimensione fissa)")
plt.axis('off')
plt.colorbar(nodes, label="Eigenvector Centrality")
plt.show()

# Opzionale: stampa eigenvector centrality ordinata
print("Eigenvector Centrality:")
for node, centrality in sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True):
    print(f"{node}: {centrality:.4f}")


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Carichiamo il dataset
dataset_path = 'dataset_con_peso.csv'
df = pd.read_csv(dataset_path)

# Costruzione del grafo
G = nx.Graph()
for _, row in df.iterrows():
    G.add_edge(row['Source'], row['Target'], weight=row['weight'])

# Troviamo la clique massima
max_clique = max(nx.find_cliques(G), key=len)

# Sottografo contenente solo la clique massima
clique_subgraph = G.subgraph(max_clique)

# Spring layout per il sottografo
pos = nx.spring_layout(clique_subgraph, seed=42)

# Disegno del grafo della clique massima
plt.figure(figsize=(8, 6))
nx.draw_networkx_nodes(clique_subgraph, pos, node_size=300, node_color='lightblue', alpha=0.9)
nx.draw_networkx_edges(clique_subgraph, pos, width=1.5, alpha=0.8)
nx.draw_networkx_labels(clique_subgraph, pos, font_size=9)

plt.title(f"Clique Massima (dimensione: {len(max_clique)})")
plt.axis('off')
plt.show()

# Stampa opzionale dei membri della clique
print("Membri della clique massima:")
for actor in max_clique:
    print(actor)


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Carichiamo il dataset
dataset_path = 'dataset_con_peso.csv'
df = pd.read_csv(dataset_path)

# Costruzione del grafo
G = nx.Graph()
for _, row in df.iterrows():
    G.add_edge(row['Source'], row['Target'], weight=row['weight'])

# Estrazione del k-core con k=15
k = 15
k_core_subgraph = nx.k_core(G, k=k)

# Verifica se il k-core non è vuoto
if len(k_core_subgraph) > 0:
    # Spring layout
    pos = nx.spring_layout(k_core_subgraph, seed=42)

    # Disegno del k-core
    plt.figure(figsize=(10, 8))
    nx.draw_networkx_nodes(k_core_subgraph, pos, node_size=250, node_color='lightgreen', alpha=0.9)
    nx.draw_networkx_edges(k_core_subgraph, pos, width=1.5, alpha=0.7)
    nx.draw_networkx_labels(k_core_subgraph, pos, font_size=8)

    plt.title(f"{k}-core del grafo (nodi con grado ≥ {k}) - {len(k_core_subgraph.nodes())} nodi")
    plt.axis('off')
    plt.show()

    # Lista dei nodi nel k-core
    print(f"Nodi nel {k}-core:")
    for node in k_core_subgraph.nodes():
        print(node)
else:
    print(f"Nessun nodo trovato nel {k}-core (grado minimo {k})")


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Caricamento dataset
dataset_path = 'dataset_con_peso.csv'
df = pd.read_csv(dataset_path)

# Costruzione del grafo
G = nx.Graph()
for _, row in df.iterrows():
    if row['weight'] > 2:
        G.add_edge(row['Source'], row['Target'], weight=row['weight'])

# Nome dell'attore da analizzare
ego_actor = 'Stan Lee'

# Verifica che Stan Lee sia presente nel grafo
if ego_actor in G.nodes():
    # Costruzione dell'ego network con archi filtrati
    ego_net = nx.ego_graph(G, ego_actor)

    # Layout
    pos = nx.spring_layout(ego_net, seed=42)

    # Disegno del grafo
    plt.figure(figsize=(10, 8))
    node_colors = ['orange' if node == ego_actor else 'lightgreen' for node in ego_net.nodes()]
    
    nx.draw_networkx_nodes(ego_net, pos, node_color=node_colors, node_size=300, alpha=0.9)
    nx.draw_networkx_edges(ego_net, pos, width=1.5, alpha=0.7)
    nx.draw_networkx_labels(ego_net, pos, font_size=9)

    plt.title("Ego Network di Stan Lee (solo archi con peso > 2)")
    plt.axis('off')
    plt.show()

    print(f"Numero di nodi nell'ego network (incluso Stan Lee): {len(ego_net.nodes())}")
    print(f"Numero di archi: {len(ego_net.edges())}")
    print("\nAttori collegati direttamente a Stan Lee (peso > 2):")
    for neighbor in ego_net.neighbors(ego_actor):
        weight = G[ego_actor][neighbor]['weight']
        print(f"{neighbor} (peso: {weight})")
else:
    print("Stan Lee non è presente nel grafo.")
